# LangChain PromptTemplate

이 노트북에서는 LangChain의 PromptTemplate 사용법을 살펴보겠습니다. PromptTemplate은 템플릿을 정의하고 변수를 동적으로 채워넣어 프롬프트를 생성하는 방법을 제공합니다.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## 기본 사용법

PromptTemplate은 문자열 템플릿과 입력 변수를 정의하여 쉽게 프롬프트를 생성할 수 있게 해줍니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI()

In [ ]:
# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

prompt

In [ ]:
# format 메소드로 변수 값 채우기
formatted_prompt = prompt.format(country="대한민국")
formatted_prompt

## LLM과 함께 사용하기

PromptTemplate은 LLM과 쉽게 연결하여 체인을 구성할 수 있습니다.

In [ ]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [ ]:
# 체인 실행
chain.invoke("대한민국").content

## PromptTemplate 객체 직접 생성하기

`from_template` 메소드 대신 객체를 직접 생성할 수도 있습니다.

In [ ]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt

In [ ]:
prompt.format(country="대한민국")

## partial_variables 사용하기

`partial_variables`를 사용하면 일부 변수의 값을 미리 지정할 수 있습니다.

In [ ]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

In [ ]:
# 원래라면 두 개의 값을 넣어야 하지만 하나만 넣어도 됨
prompt.format(country1="대한민국")

## partial 메소드 사용하기

`partial` 메소드를 사용하면 기존의 프롬프트 템플릿에서 일부 변수 값을 고정한 새로운 템플릿을 만들 수 있습니다.

In [ ]:
# partial 메소드로 country2 값을 변경
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

In [ ]:
prompt_partial.format(country1="대한민국")

In [ ]:
# 체인 생성 및 실행
chain = prompt_partial | llm
chain.invoke("대한민국").content

In [ ]:
# 값을 다른 것으로 지정해도 실행 가능
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

## 함수를 partial_variables로 사용하기

partial_variables에는 값 대신 함수를 전달할 수도 있습니다.

In [ ]:
from datetime import datetime

# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [ ]:
# 함수를 partial_variables로 사용
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # 함수 자체를 전달
    },
)

prompt

In [ ]:
# 템플릿 포맷팅 - 함수가 호출되어 결과가 사용됨
prompt.format(n=3)

In [ ]:
# 체인 생성 및 실행
chain = prompt | ChatOpenAI(model_name="gpt-4o", temperature=0)
print(chain.invoke(3).content)

In [ ]:
# 딕셔너리로 값을 수정해 사용 가능
print(chain.invoke({"today": "Jan 02", "n": 3}).content)